In [0]:
import pandas as pd
import numpy as np
import os, time, pickle
import datetime as dt



In [0]:
Sw_Meetdata = []
Sw_Samen = []
Sw_Bouwstenen = []

f = ['Switch01_Data.xlsx', 'Switch02_Data.xlsx','Switch03_Data.xlsx', 'Switch04_Data.xlsx', 'Switch05_Data.xlsx']

ControlPosition = []
SteeringCommand = []
MotorCurrentActive = []

Sw_Bouwstenen.append(pd.read_excel(f[1],sheet_name="Bouwstenen"))
Sw_Bouwstenen[-1] = Sw_Bouwstenen[-1].dropna(axis = 'columns', how = 'all')

ControlPosition.append(Sw_Bouwstenen[0].iloc[:,0:2])
#Filtering NaN
ControlPosition[0] = ControlPosition[0].dropna(axis = 'index', how = 'all')

SteeringCommand.append(Sw_Bouwstenen[0].iloc[:,2:4])
SteeringCommand[0] = SteeringCommand[0].dropna(axis = 'index', how = 'all')

MotorCurrentActive.append(Sw_Bouwstenen[0].iloc[:,4:6])
MotorCurrentActive[0] = MotorCurrentActive[0].dropna(axis = 'index', how = 'all')

#Vectors d'errors
#Initialized to false, so only modified to true in case of error/failure
StartsTooLate = SteeringCommand[0].shape[0]*[0]
ReturnsToSame = SteeringCommand[0].shape[0]*[0]
StopsDuringMovement = SteeringCommand[0].shape[0]*[0]
NonsenseOrder = SteeringCommand[0].shape[0]*[0]
SwitchNotReady = SteeringCommand[0].shape[0]*[0]
MovementWithoutOrder = SteeringCommand[0].shape[0]*[0]
FailOutDetector = SteeringCommand[0].shape[0]*[0]
FailRLDetector = SteeringCommand[0].shape[0]*[0]


El següent codi mira cada ordre de steering que s'ha donat i comprova:

*   Si els moviments de l'agulla són correctes, és a dir, que se satisfà aquest cicle:

 *   R
 *   Steer to L
 *   Out
 *   L
 *   Steer to R
 *   Out
 *   R

*   Si el motor tarda massa a posar-se en marxa
*   Si el motor es para durant el moviment d'una banda a l'altra

In [0]:
j = 0
index = 0
m = 0
length = SteeringCommand[0].shape[0]
for i in range(length):
  #Quan es dona l'ordre
  comActualTime = SteeringCommand[0].SteeringCommandDate[i]
  comActual = SteeringCommand[0].SteeringCommand[i]

  PosTime = ControlPosition[0].ControlPositionDate[j]
  while PosTime < comActualTime:
    j = j+1
    PosTime = ControlPosition[0].ControlPositionDate[j]
  
  if j != 0:
    prevPos = ControlPosition[0].ControlPosition[j-1]
  pos = ControlPosition[0].ControlPosition[j]
  nextPos = ControlPosition[0].ControlPosition[j+1]
  
  #Si l'ordre es anar de L a R
  if comActual == "Sturing Rechts":
    if j != 0 and prevPos != "Links in controle":
      if prevPos == "Rechts in controle": #si estava a la dreta no te sentit moure a la dreta
        NonsenseOrder[i] = 1
      else:  #encara no havia acabat o comenca abans de l'ordre
        SwitchNotReady[i] = 1
        
    elif pos != "Uit controle":
      FailOutDetector[i] = 1
      #no detecta be quan esta out (perque per a tornar a detectar el mateix ha de
      #deixar de detectarlo i per a detectar el contrari ha d'haver estat out)
        
    elif nextPos != "Rechts in controle":
      if nextPos == "Links in controle": #l'agulla ha tornat a l'origen
        ReturnsToSame[i] = 1
      else: #el detector links o rechts no funciona be
        FailRLDetector[i] = 1
      
    elif j < length-2 and ControlPosition[0].ControlPositionDate[j+2] < SteeringCommand[0].SteeringCommandDate[i+1]:
      #abans de l'ordre seguent hi ha mes moviment
      MovementWithoutOrder[i] = 1
      #interessa que fa? la lia o torna a la mateixa pos abans que passe el tren?
      
    #else   Esta tot perfecte
      
  else:
    if j != 0 and prevPos != "Rechts in controle":
      if prevPos == "Links in controle":
        NonsenseOrder[i] = 1
      else:
        SwitchNotReady[i] = 1
    elif pos != "Uit controle":
      FailOutDetector[i] = 1
    elif nextPos != "Links in controle":
      if nextPos == "Rechts in controle":
        ReturnsToSame[i] = 1
      else:
        FailRLDetector[i] = 1
    elif j < length-2 and ControlPosition[0].ControlPositionDate[j+2] < SteeringCommand[0].SteeringCommandDate[i+1]:
      MovementWithoutOrder[i] = 1
    #else   Esta tot perfecte
  
  #Quan s'activa el motor despres de l'ordre?
  motStartTime = MotorCurrentActive[0].MotorCurrentActiveDate[index]
  while motStartTime < comActualTime and index < MotorCurrentActive[0].shape[0]-1:
    index = index + 1
    if MotorCurrentActive[0].MotorCurrentActive[index] == "Motorstroom Actief":
      #Quan s'activa el motor despres de l'ordre
      motStartTime = MotorCurrentActive[0].MotorCurrentActiveDate[index]
  
  #El switch comenca a moure's massa tard? Si = ERROR
  tol = 2
  if motStartTime - comActualTime > dt.timedelta(seconds=tol):
    StartsTooLate[i] = 1

  #Quan es para el motor?
  if index < MotorCurrentActive[0].shape[0]-1:
    k = index + 1
    motEndTime = MotorCurrentActive[0].MotorCurrentActiveDate[k]
    while motEndTime == "Motorstroom Actief":
      k = k + 1
      motEndTime = MotorCurrentActive[0].MotorCurrentActiveDate[k] 
    
  #Quan arriba al final la palanca?
  finalMov = ControlPosition[0].ControlPosition[m]
  finalMovTime = ControlPosition[0].ControlPositionDate[m]
  while finalMovTime < comActualTime or finalMov == "Uit controle":
    m = m + 1
    finalMov = ControlPosition[0].ControlPosition[m]
    finalMovTime = ControlPosition[0].ControlPositionDate[m]

  #El switch es para enmig del moviment? Si = ERROR
  TolTemps = dt.timedelta(seconds=0.6)
  if motEndTime+TolTemps < finalMovTime:
    StopsDuringMovement[i] = 1



In [49]:
for i in range(SteeringCommand[0].shape[0]):
  comActualTime = SteeringCommand[0].SteeringCommandDate[i]
  comActual = SteeringCommand[0].SteeringCommand[i]
  
  print('Order ' + str(i+1) + ': ' + str(comActual) + ' at time ' + str(comActualTime))
  
  BoolError = 0
  
  if NonsenseOrder[i] == 1:
    print('The order is nonsense, the switch is in the requested position already.')
    BoolError = 1
    
  if SwitchNotReady[i] == 1:
    print('The switch is not ready for a new order, it is still moving.')
    BoolError = 1
    
  if StartsTooLate[i] == 1:
    print('The motor starts too late after the order.')
    BoolError = 1
    
  if StopsDuringMovement[i] == 1:
    print('The motor stops during the movement.')
    BoolError = 1
    
  if ReturnsToSame[i] == 1:
    print('The switch has returned to the initial position')
    BoolError = 1
    
  if FailOutDetector[i] == 1:
    print('The detector for Out has failed.')
    BoolError = 1
    
  if FailRLDetector[i] == 1:
    print('The detector on the Right or Left side has failed.')
    BoolError = 1
    
  if BoolError == 0:
    print('___SUCCESS___')
  
  print()

Order 1: Sturing Rechts at time 2019-04-01 04:58:20.454000
___SUCCESS___

Order 2: Sturing Links at time 2019-04-01 05:06:01.754000
___SUCCESS___

Order 3: Sturing Rechts at time 2019-04-01 06:04:24.305000
___SUCCESS___

Order 4: Sturing Links at time 2019-04-01 06:06:00.970000
___SUCCESS___

Order 5: Sturing Rechts at time 2019-04-01 07:16:25.097000
___SUCCESS___

Order 6: Sturing Links at time 2019-04-01 07:19:42.997000
___SUCCESS___

Order 7: Sturing Rechts at time 2019-04-01 07:45:44.261000
___SUCCESS___

Order 8: Sturing Links at time 2019-04-01 07:47:15.411000
___SUCCESS___

Order 9: Sturing Rechts at time 2019-04-01 08:54:01.154000
___SUCCESS___

Order 10: Sturing Links at time 2019-04-01 09:05:16.904000
___SUCCESS___

Order 11: Sturing Rechts at time 2019-04-01 09:15:14.604000
___SUCCESS___

Order 12: Sturing Links at time 2019-04-01 09:17:01.354000
___SUCCESS___

Order 13: Sturing Rechts at time 2019-04-01 09:49:54.505000
___SUCCESS___

Order 14: Sturing Links at time 2019-04-

El següent codi comprova que no passa cap tren mentre el switch s'està movent

OBS: no funciona per al dataset2 perque no hi ha dades de trens

In [24]:
ControlOutTime = []
SectionOccupation = []

Sw_Samen.append(pd.read_excel(f[0],sheet_name="Samenvattende Waarde"))
Sw_Samen[-1] = Sw_Samen[-1].dropna(axis = 'columns', how = 'all')

ControlOutTime.append(Sw_Samen[0].iloc[:,28:30])
ControlOutTime[0] = ControlOutTime[0].dropna(axis = 'index', how = 'all')

SectionOccupationTime.append(Sw_Samen[0].iloc[:,30:32])
SectionOccupationTime[0] = SectionOccupationTime[0].dropna(axis = 'index', how = 'all')

IsTrainAbove = [0]*ControlOutTime[0].shape[0]

toler = 10
toler = dt.timedelta(seconds=toler)

#for every moment in which the switch is out
for i in range(ControlOutTime[0].shape[0]):
  outStart = ControlOutTime[0].ControlOutTimeDate[i]
  outDur = ControlOutTime[0].ControlOutTime[i]
  
  for j in range(SectionOccupation[0].shape[0]):
    TrainStartTime = SectionOccupation[0].SectionOccupationTimeDate[j]
    TrainDuration = SectionOccupation[0].SectionOccupationTime[j]
    TrainDuration = dt.timedelta(seconds=TrainDuration)

    if not(TrainStartTime > outStart+outDur+toler or TrainStartTime+TrainDuration+toler < outStart):
      IsTrainAbove[i] = 1
      break
  

NameError: ignored